In [2]:
import torch 
import torch.nn as nn 
from model import Transformer
from config import get_config,get_weights_file_path
from train import get_model,get_ds,greedy_decode
import altair as alt 
import pandas as pd 
import numpy as np 
import warnings 
warnings.filterwarnings('ignore')

In [3]:
# define the device 
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device {device}")

using device cpu


In [4]:
config = get_config()
train_dataloader,val_dataloader,vocab_src,vocab_tgt = get_ds(config)
model = get_model(config,vocab_src.get_vocab_size(),vocab_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = get_weights_file_path(config,f"01")
state=torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])


Max length of source sentence: 309
Max length of target sentence: 274


FileNotFoundError: [Errno 2] No such file or directory: 'opus_books_weights/tmodel_01.pt'

In [5]:
def load_next_batch():
    batch = next(iter(val_dataloader))
    encoder_input = batch['encoder_input'].to(device)
    encoder_mask = batch['encoder_mask'].to(device)
    decoder_input = batch['decoder_input'].to(device)
    
    encoder_input_tokens = [vocab_src.id_to_token(idx) for idx in encoder_input[0].cpu().numpy()]
    decoder_input_tokens = [vocab_tgt.id_to_token(idx) for idx in decoder_input[0].cpu().numpy()]
    
    model_out = greedy_decode(model,encoder_input,encoder_mask, vocab_src,vocab_tgt,config['seq_len'],device)
    
    return batch,encoder_input,decoder_input_tokens